<a href="https://colab.research.google.com/github/gsdev42/Medical-question-summarization-/blob/main/Fine__tuning_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%env PYTHONPATH=

env: PYTHONPATH=


In [3]:
!pip install virtualenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 23.0 MB/s eta 0:00:00


In [4]:
!virtualenv myenv

created virtual environment CPython3.10.12.final.0-64 in 1557ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.5.1, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [5]:
!pip3 install accelerate>=0.21.0

In [6]:
!pip install -U datasets
!pip install -U transformers
!pip install torch
!pip install sacrebleu
!pip install h5py typing-extensions wheel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [7]:
from datasets import load_dataset
dataset = load_dataset("albertvillanova/meqsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
dataset['train'][1]['CHQ'][:100]

'who makes bromocriptine\ni am wondering what company makes the drug bromocriptine, i need it for a ma'

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['CHQ', 'Summary', 'File'],
        num_rows: 1000
    })
})

In [10]:
split_dataset = dataset['train'].train_test_split(test_size=0.2)

In [11]:
print(split_dataset)


DatasetDict({
    train: Dataset({
        features: ['CHQ', 'Summary', 'File'],
        num_rows: 800
    })
    test: Dataset({
        features: ['CHQ', 'Summary', 'File'],
        num_rows: 200
    })
})


In [12]:
split_dataset['train']

Dataset({
    features: ['CHQ', 'Summary', 'File'],
    num_rows: 800
})

In [19]:
from transformers import BartTokenizer

# Initialize the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Tokenize the inputs and outputs
def preprocess_data(examples):
    inputs = examples['CHQ']
    targets = examples['Summary']
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=1024, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_train_dataset = (split_dataset['train']).map(preprocess_data, batched=True)
tokenized_test_dataset = (split_dataset['test']).map(preprocess_data, batched=True)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [20]:
import torch
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Load the pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [22]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',             # Output directory for model checkpoints
    evaluation_strategy='epoch',        # Evaluation strategy to use after each epoch
    save_strategy='epoch',              # Save strategy to use after each epoch
    learning_rate=2e-5,                 # Learning rate
    per_device_train_batch_size=4,      # Batch size for training
    per_device_eval_batch_size=4,       # Batch size for evaluation
    num_train_epochs=3,                 # Number of training epochs
    weight_decay=0.01,                  # Weight decay
    logging_dir='./logs',               # Directory for storing logs
    logging_steps=10,                   # Log every 10 steps
    save_total_limit=3,                 # Limit the total amount of checkpoints
    load_best_model_at_end=True,        # Load the best model at the end of training
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    tokenizer=tokenizer,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.162100,1.254671
2,0.730800,1.408128
3,0.558100,1.219542


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=600, training_loss=0.7762194347381591, metrics={'train_runtime': 424.1342, 'train_samples_per_second': 5.659, 'train_steps_per_second': 1.415, 'total_flos': 770007819190272.0, 'train_loss': 0.7762194347381591, 'epoch': 3.0})

In [27]:
model.save_pretrained('./fine-tuned-bart')
tokenizer.save_pretrained('./fine-tuned-bart')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine-tuned-bart/tokenizer_config.json',
 './fine-tuned-bart/special_tokens_map.json',
 './fine-tuned-bart/vocab.json',
 './fine-tuned-bart/merges.txt',
 './fine-tuned-bart/added_tokens.json')

In [28]:
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_l

In [30]:
ref_summary_1="""How can I find physician(s) or hospital(s) who specialize in pheochromocytomas?"""
ref_summary_2="""Where can I find information on mecp2 gene mutation and ppm-x syndrome?"""

In [31]:
sample1="""I understand that you cannot provide opinions,nor suggest, any type of therpy.
My son (dob-[DATE]), was called back to the hospital four days after birth due to a abnormal PKU, which suggested adrenal hyperplasia.
After test the endocrine physians felt he did not have the condition. Several years later he began having severe headaches preceded by violent
vomiting epidodes that lasted for hours. When he was admitted to [LOCATION],  he was suffering from bleeding esphogal ulcers,  and severe
dehydration and, malnutrician. His urine (24 hr), blood plasma,& VMA,showed increased values of catacholomines.He also demostrated intermittent
hypertension, and tachcardia through out the day (he had been diagnoised with ADHD several years earlier). His MIGB clearly showed bifocal uptake
in both adrenal glands. The mri did not show tumors at that time ([DATE]. He has since been diagnoised with antral polys (biosphy's taken were misplaced,
 and the gastrologist would not redo the test), aberrant right subclavian artery,  esphogas pushed to the side, low oxygen intake, high levels of gastrin,
  cyclic vomiting syndrome, severe sinus attacks that come and go, without infection present.
His symptons that I have not already include heat intolerence, pallor, poor appetite with little weight gain, anxiety, tremor of hands, and stomach pain.
After his mri, which I was told would take about 45mins, took over 5 hrs. Since then we have been having no luck with finding doctors that will take my
son as a patient. Are you able to suggest a hospital that is knowlegable with pheochromocytomas? My son's father is mia, but during our 7 yrs together,
  he also suffered severe headaches, and intermittent hypertension. He never followed up with a physian. Any help would be appreciated. I do not know
  what to do, and I do not want my son to end up a statistic.  Last night his BB was 180/115, with HR-50, half hour later his BP was 68/44 with HR of 122.
   I fear he is going to go in shoc"""

sample2="""SUBJECT: ClinicalTrials.gov - Question - general information
MESSAGE: I have a [AGE] and a [AGE] boys who have been diagnosed with the mecp2 gene mutation and yesterday my neurologist was also commenting
 on the possibility of ppm-x I am very concerned that no-one in this field can he me. I have been seen durastic changes that have no changes
 like any of the other characteristics that have been reported.I am desperate to keep them safe and comfortable for the rest of life and could
 realy use help.  [NAME]   [CONTACT] please help me with any information it would be appreciated more than you know."""


In [32]:
# Format the input text as required by the model
input_text = f"assign tag: {ref_summary_1} {sample1}"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=512).input_ids
input_ids = input_ids.to(device)

In [33]:
# Generate the prediction
outputs = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)

# Decode the prediction
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Tags:", prediction)

Generated Tags: Where can I find physician(s) or hospital (s) who specialize in pheochromocytomas?


In [34]:
# Format the input text as required by the model
input_text = f"assign tag: {ref_summary_2} {sample2}"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=512).input_ids
input_ids = input_ids.to(device)

In [35]:
# Generate the prediction
outputs = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)

# Decode the prediction
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Tags:", prediction)

Generated Tags: Where can I find information on mecp2 gene mutation and ppm-x syndrome?


In [37]:
summ1='''Where can I find physician(s) or hospital (s) who specialize in pheochromocytomas?'''
summ2='''Where can I find information on mecp2 gene mutation and ppm-x syndrome?'''

In [40]:
import sacrebleu

reference_summaries = [
    ref_summary_1,
    ref_summary_2

]

generated_summaries_BART = [
    summ1,
    summ2
]

# Flatten the reference summaries for sacrebleu
flattened_references = [[ref] for refs in reference_summaries for ref in refs]

# Calculate BLEU for T5
bleu_BART = sacrebleu.corpus_bleu(generated_summaries_BART, flattened_references)
print(f"BART BLEU score: {bleu_BART.score}")

BART BLEU score: 3.211547431691929
